In [2]:
%pylab inline
%config IPCompleter.greedy=True
import requests # interaction with the web
import os  #  file system operations
import yaml # human-friendly data format
import re  # regular expressions
import pandas as pd # pandas... the best time series library out there
import datetime as dt # date and time functions
import io
import time
import sqlite3
from common_fuctions import cookie_crumb

Populating the interactive namespace from numpy and matplotlib


Список бумаг с которыми мы будем работать:

In [3]:
SP_SYMBOLS =['CL=F','NG=F','CT=F','KC=F','CT=F']
#SP_SYMBOLS =['GC=F','CL=F']

Период для работы:

In [4]:
first_year = 2000
last_year = 2021

Файл БД

In [5]:
base = 'invest2.db'

Функция получения цен за год:

In [6]:
def getprices(symbol,year,crumb,cookie):
    sDate = (year,1,1)
    eDate = (year,12,31)
    dt.datetime(*sDate).timestamp() 
    data = (int(dt.datetime(*sDate).timestamp()),
        int(dt.datetime(*eDate).timestamp()), 
        crumb)
    url = "https://query1.finance.yahoo.com/v7/finance/download/"+symbol+"?period1={0}&period2={1}&interval=1d&events=history&crumb={2}".format(*data)
    data = requests.get(url, cookies={'B':cookie}, headers={'User-Agent': 'Mozilla/5.0'})
    buf = io.StringIO(data.text) # create a buffer
    try:
        #df = pd.read_csv(buf,index_col=0) # convert to pandas DataFrame
        df = pd.read_csv(buf)
    except:
        df =pd.DataFrame()
    return df

Функция вставки истоических значений в базу

In [7]:
def insert_history(base,symbol,date,op,high,low,close,volume):
    conn = sqlite3.connect(base)
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO comodity_history (SYMBOL,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,UNIQ) VALUES (?,?,?,?,?,?,?,?);",
                      (symbol,date,op,high,low,close,volume,symbol+date,))
        conn.commit()
    except:
        pass
    finally:
        conn.close()

# Основной цикл

In [8]:
cookie, crumb = cookie_crumb()

In [9]:
%%time
i=0
total=len(SP_SYMBOLS)
DATA = pd.DataFrame() 
for symbol in SP_SYMBOLS:
    for year in range(first_year,last_year+1):
        df = getprices(symbol,year,crumb,cookie)
        df = df.dropna()
        time.sleep(1)
        if ((len(df) > 10)):            
            k=0
            for k  in range(0,(len(df))):
                date = list(df[k:k+1]['Date'])[0]
                op = list(df[k:k+1]['Open'])[0]
                high = list(df[k:k+1]['High'])[0]
                low = list(df[k:k+1]['Low'])[0]
                close = list(df[k:k+1]['Close'])[0]
                volume = list(df[k:k+1]['Volume'])[0]
                try:
                    insert_history(base,symbol,date,op,high,low,close,volume)
                except:
                    pass
     
    i=i+1;
    print(str(int((i/total)*100))+'% Copmplete')        

20% Copmplete
40% Copmplete
60% Copmplete
80% Copmplete
100% Copmplete
CPU times: user 15.2 s, sys: 847 ms, total: 16 s
Wall time: 2min 44s
